## **Load Dataset from Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/Dataset/datacar.zip"

Streaming output truncated to the last 5000 lines.
  inflating: vehicles/2734.png       
  inflating: vehicles/2735.png       
  inflating: vehicles/2736.png       
  inflating: vehicles/2737.png       
  inflating: vehicles/2738.png       
  inflating: vehicles/2739.png       
  inflating: vehicles/274.png        
  inflating: vehicles/2740.png       
  inflating: vehicles/2741.png       
  inflating: vehicles/2742.png       
  inflating: vehicles/2743.png       
  inflating: vehicles/2744.png       
  inflating: vehicles/2745.png       
  inflating: vehicles/2746.png       
  inflating: vehicles/2747.png       
  inflating: vehicles/2748.png       
  inflating: vehicles/2749.png       
  inflating: vehicles/275.png        
  inflating: vehicles/2750.png       
  inflating: vehicles/2751.png       
  inflating: vehicles/2752.png       
  inflating: vehicles/2753.png       
  inflating: vehicles/2754.png       
  inflating: vehicles/2755.png       
  inflating: vehicles/2756.png       

## **Import Library and Show Dataset**

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import glob
from skimage.feature import hog
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
cars_path = "/content/vehicles"
nocars_path = "/content/non-vehicles"

cars = glob.glob(f"{cars_path}/*.*", recursive=True)
notcars = glob.glob(f"{nocars_path}/*.*", recursive=True)

In [5]:
print('cars contain {} images'.format(len(cars)))
print('nocars contain {} images'.format(len(notcars)))

cars contain 8792 images
nocars contain 8968 images


## **Feature Config for HOG**

In [6]:
config = {}
config['color_space'] = 'HLS'
config['orient'] = 9
config['pix_per_cell'] = 8
config['cell_per_block'] = 2
config['hog_channel'] = 'ALL'
config['spatial_size'] = (16, 16)
config['hist_bins'] = 32
config['spatial_feat'] = True
config['hist_feat'] = True
config['hog_feat'] = True

In [7]:
color_space = config['color_space']
orient = config['orient']
pix_per_cell = config['pix_per_cell']
cell_per_block = config['cell_per_block']
hog_channel = config['hog_channel']
spatial_size = config['spatial_size']
hist_bins = config['hist_bins']
spatial_feat = config['spatial_feat']
hist_feat = config['hist_feat']
hog_feat = config['hog_feat']

## **Main Function for Feature Extraction**

In [ ]:
# Mengonversi citra dari satu ruang warna ke ruang warna lainnya, seperti RGB ke YCrCb, HSV, HLS, LUV, atau YUV.
def convert_color(img, color_space='YCrCb'):
    if color_space == 'RGB':
        return np.copy(img)
    elif color_space == 'YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    elif color_space == 'HSV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    elif color_space == 'HLS':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    elif color_space == 'LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    elif color_space == 'YUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YUV)

# Menghitung Histogram of Oriented Gradients (HOG)
def get_hog_features(img, orient, pix_per_cell, cell_per_block,
                        vis=False, feature_vec=True):
    if vis == True:
        features, hog_image = hog(img, orientations=orient,
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  transform_sqrt=True,
                                  visualize=vis, feature_vector=feature_vec)
        return features, hog_image
    else:
        features = hog(img, orientations=orient,
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       transform_sqrt=True,
                       visualize=vis, feature_vector=feature_vec)

        return features

# Menghitung fitur spasia
def bin_spatial(img, size=(32, 32)):
    features = cv2.resize(img, size).ravel()

    return features

# Menghitung histogram warna
def color_hist(img, nbins=32, bins_range=(0, 256)):
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))

    return hist_features

# Menggabungkan semua fitur
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9,
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    features = []
    for file in imgs:
        file_features = []

        image = mpimg.imread(file)

        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel],
                                        orient, pix_per_cell, cell_per_block,
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient,
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)

            file_features.append(hog_features)
        features.append(np.concatenate(file_features))

    return features

# Membuat daftar jendela geser
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None],
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]

    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]

    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))

    nx_windows = np.int(xspan/nx_pix_per_step) - 1
    ny_windows = np.int(yspan/ny_pix_per_step) - 1

    window_list = []

    for ys in range(ny_windows):
        for xs in range(nx_windows):
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            window_list.append(((startx, starty), (endx, endy)))

    return window_list

# Menggambar box pada suatu citra berdasarkan daftar box
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):

    imcopy = np.copy(img)

    for bbox in bboxes:
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)

    return imcopy

# Menghitung fitur dari suatu citra dari hasil ekstraksi
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9,
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):

    img_features = []

    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)

    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        img_features.append(spatial_features)
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        img_features.append(hist_features)
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel],
                                    orient, pix_per_cell, cell_per_block,
                                    vis=False, feature_vec=True))
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient,
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)

        img_features.append(hog_features)

    return np.concatenate(img_features)

# Menggunakan model yang telah dilatih untuk melakukan deteksi
def search_windows(img, windows, clf, scaler, color_space='RGB',
                    spatial_size=(32, 32), hist_bins=32,
                    hist_range=(0, 256), orient=9,
                    pix_per_cell=8, cell_per_block=2,
                    hog_channel=0, spatial_feat=True,
                    hist_feat=True, hog_feat=True):

    on_windows = []

    for window in windows:
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))
        features = single_img_features(test_img, color_space=color_space,
                            spatial_size=spatial_size, hist_bins=hist_bins,
                            orient=orient, pix_per_cell=pix_per_cell,
                            cell_per_block=cell_per_block,
                            hog_channel=hog_channel, spatial_feat=spatial_feat,
                            hist_feat=hist_feat, hog_feat=hog_feat)

        test_features = scaler.transform(np.array(features).reshape(1, -1))

        prediction = clf.predict(test_features)

        if prediction == 1:
            on_windows.append(window)

    return on_windows

In [ ]:
# Feature extraction from dataset
car_features = extract_features(cars, color_space=color_space, orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                                hog_channel=hog_channel, spatial_size=spatial_size, hist_bins=hist_bins, spatial_feat=spatial_feat, hist_feat=hist_feat,
                                hog_feat=hog_feat)
nocar_features = extract_features(notcars, color_space=color_space, orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                                hog_channel=hog_channel, spatial_size=spatial_size, hist_bins=hist_bins, spatial_feat=spatial_feat, hist_feat=hist_feat,
                                hog_feat=hog_feat)


# Merge feature and labeling for training
# 0 untuk negatif dan 1 untuk positif
features = np.vstack((car_features, nocar_features)).astype(np.float64)
labels = np.hstack((np.ones(len(cars)), np.zeros(len(notcars))))

## **Data Preprocessing**

In [ ]:
# Scaling and Split Dataset
Xscaler = StandardScaler()
Xscaler.fit(features)
scaled_features = Xscaler.transform(features)
X_train, X_valid, y_train, y_valid = train_test_split(scaled_features, labels, test_size=0.2, random_state=42)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2, random_state=42)

Xscaler = StandardScaler()
Xscaler.fit(X_train)

X_train_scaled = Xscaler.transform(X_train)
X_valid_scaled = Xscaler.transform(X_valid)

## **Model Training with SVC**

In [ ]:
svc = LinearSVC()
svc.fit(X_train_scaled, y_train)
print('The validation accuracy is {} %'.format((svc.score(X_valid_scaled, y_valid))))

The validation accuracy is 0.986204954954955 %


## **Save Model Function**

In [ ]:
def save_trained_model(pickle_file, classifier, Xscaler, config):
    try:
        with open(pickle_file, 'wb') as pfile:
            pickle.dump(
                {   'classifier':classifier,
                    'scaler':Xscaler,

                    'color_space': config['color_space'],
                    'orient': config['orient'],
                    'pix_per_cell': config['pix_per_cell'],
                    'cell_per_block': config['cell_per_block'],
                    'hog_channel': config['hog_channel'],
                    'spatial_size': config['spatial_size'],
                    'hist_bins': config['hist_bins'],
                    'spatial_feat': config['spatial_feat'],
                    'hist_feat': config['hist_feat'],
                    'hog_feat': config['hog_feat']
                },
                pfile, pickle.HIGHEST_PROTOCOL)
        print('Classifier saved to {}'.format(pickle_file))
    except Exception as e:
        print('Failed to save classifier to {} : {}'.format(pickle_file, e))
        raise

def load_trained_model(pickle_file):
    try:
        with open(pickle_file, 'rb') as f:
            classifier = pickle.load(f)
        return classifier
        print('Trained model {} is loaded !'.format(pickle_file))

    except Exception as e:
        print('Failed to load {}: {}!'.format(pickle_file, e))
        raise

## **Save Model to Local**

In [ ]:
import os

save_dir = './trained_models/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

save_trained_model(os.path.join(save_dir, 'SVM_HLS_1.p'), svc, Xscaler, config)

Classifier saved to ./trained_models/SVM_HLS_1.p
